In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-ratings/movie_ratings_df.csv


In [2]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 KB 8.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b4d1deb5367eb059995f512731e0d031b97150557250e53ffc674cb7ebdf00e5
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [3]:
#import spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('recom_sys').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/26 06:59:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#import libs
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import IndexToString
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
#read the dataset
df = spark.read.csv('/kaggle/input/movie-ratings/movie_ratings_df.csv', header=True, inferSchema=True)
df.show(1)


+------+------------+------+
|userId|       title|rating|
+------+------------+------+
|   196|Kolya (1996)|     3|
+------+------------+------+
only showing top 1 row



In [6]:
#shape of data
print(((df.count()), len(df.columns)))

(100000, 3)


In [7]:
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)



In [8]:
df.orderBy(rand()).show(10,False)

+------+--------------------------------------------------+------+
|userId|title                                             |rating|
+------+--------------------------------------------------+------+
|846   |Demolition Man (1993)                             |3     |
|379   |Liar Liar (1997)                                  |3     |
|328   |Bridge on the River Kwai, The (1957)              |4     |
|537   |Bound (1996)                                      |3     |
|889   |Sneakers (1992)                                   |4     |
|854   |People vs. Larry Flynt, The (1996)                |4     |
|393   |Austin Powers: International Man of Mystery (1997)|3     |
|130   |Nell (1994)                                       |4     |
|833   |Abyss, The (1989)                                 |2     |
|76    |Welcome to the Dollhouse (1995)                   |4     |
+------+--------------------------------------------------+------+
only showing top 10 rows



In [9]:
#checking the ratings per user
df.groupby('userid').count().orderBy('count', ascending=False).show(10,False)

+------+-----+
|userid|count|
+------+-----+
|405   |737  |
|655   |685  |
|13    |636  |
|450   |540  |
|276   |518  |
|416   |493  |
|537   |490  |
|303   |484  |
|234   |480  |
|393   |448  |
+------+-----+
only showing top 10 rows



In [10]:
#checking the ratings per user
df.groupby('userid').count().orderBy('count', ascending=True).show(10,False)

+------+-----+
|userid|count|
+------+-----+
|19    |20   |
|143   |20   |
|309   |20   |
|34    |20   |
|202   |20   |
|732   |20   |
|441   |20   |
|685   |20   |
|824   |20   |
|631   |20   |
+------+-----+
only showing top 10 rows



In [11]:
#movies that is most rated
df.groupby('Title').count().orderBy('count', ascending=False).show(10, False)

+-----------------------------+-----+
|Title                        |count|
+-----------------------------+-----+
|Star Wars (1977)             |583  |
|Contact (1997)               |509  |
|Fargo (1996)                 |508  |
|Return of the Jedi (1983)    |507  |
|Liar Liar (1997)             |485  |
|English Patient, The (1996)  |481  |
|Scream (1996)                |478  |
|Toy Story (1995)             |452  |
|Air Force One (1997)         |431  |
|Independence Day (ID4) (1996)|429  |
+-----------------------------+-----+
only showing top 10 rows



In [12]:
#least rated movie
df.groupby('Title').count().orderBy('count', ascending=True).show(10, False)

+------------------------------------------+-----+
|Title                                     |count|
+------------------------------------------+-----+
|Modern Affair, A (1995)                   |1    |
|Tigrero: A Film That Was Never Made (1994)|1    |
|JLG/JLG - autoportrait de d�cembre (1994) |1    |
|Target (1995)                             |1    |
|Vie est belle, La (Life is Rosey) (1987)  |1    |
|Fear, The (1995)                          |1    |
|Next Step, The (1995)                     |1    |
|Mad Dog Time (1996)                       |1    |
|Leopard Son, The (1996)                   |1    |
|Lashou shentan (1992)                     |1    |
+------------------------------------------+-----+
only showing top 10 rows



In [13]:
stringIndexer = StringIndexer(inputCol='title', outputCol='title_new')
model = stringIndexer.fit(df)
indexed = model.transform(df)

In [14]:
indexed.show(10)

+------+------------+------+---------+
|userId|       title|rating|title_new|
+------+------------+------+---------+
|   196|Kolya (1996)|     3|    287.0|
|    63|Kolya (1996)|     3|    287.0|
|   226|Kolya (1996)|     5|    287.0|
|   154|Kolya (1996)|     3|    287.0|
|   306|Kolya (1996)|     5|    287.0|
|   296|Kolya (1996)|     4|    287.0|
|    34|Kolya (1996)|     5|    287.0|
|   271|Kolya (1996)|     4|    287.0|
|   201|Kolya (1996)|     4|    287.0|
|   209|Kolya (1996)|     4|    287.0|
+------+------------+------+---------+
only showing top 10 rows



In [15]:
indexed.groupby('title_new').count().orderBy('count',ascending=False).show(10,False)

+---------+-----+
|title_new|count|
+---------+-----+
|0.0      |583  |
|1.0      |509  |
|2.0      |508  |
|3.0      |507  |
|4.0      |485  |
|5.0      |481  |
|6.0      |478  |
|7.0      |452  |
|8.0      |431  |
|9.0      |429  |
+---------+-----+
only showing top 10 rows



In [16]:
traindf,testdf = indexed.randomSplit([0.75,0.25], seed=1)


In [17]:
print(((traindf.count()), len(traindf.columns)))

(74839, 4)


In [18]:
print(((testdf.count()), len(testdf.columns)))

(25161, 4)


In [19]:
rec= ALS(maxIter=10,regParam=0.01, userCol='userId', itemCol='title_new',ratingCol='rating', nonnegative=True,coldStartStrategy="drop")
rec_model = rec.fit(traindf)

In [20]:
predicted_ratings = rec_model.transform(testdf)

In [21]:
predicted_ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- title_new: double (nullable = false)
 |-- prediction: float (nullable = false)



In [22]:
predicted_ratings.show(10, False)

+------+---------------------------------------------+------+---------+----------+
|userId|title                                        |rating|title_new|prediction|
+------+---------------------------------------------+------+---------+----------+
|148   |Amadeus (1984)                               |1     |50.0     |3.4869342 |
|148   |Beauty and the Beast (1991)                  |4     |114.0    |3.858587  |
|148   |Brazil (1985)                                |4     |109.0    |4.90611   |
|148   |Free Willy (1993)                            |1     |761.0    |3.1405175 |
|148   |Gone with the Wind (1939)                    |5     |162.0    |4.202469  |
|148   |Good, The Bad and The Ugly, The (1966)       |2     |223.0    |3.837092  |
|148   |Homeward Bound: The Incredible Journey (1993)|1     |533.0    |4.167219  |
|148   |James and the Giant Peach (1996)             |5     |260.0    |4.9838953 |
|148   |My Left Foot (1989)                          |5     |278.0    |3.073872  |
|148

In [23]:
evaluator = RegressionEvaluator(metricName='rmse', predictionCol='prediction',labelCol='rating')

In [24]:
rmse=evaluator.evaluate(predicted_ratings)

In [25]:
print(rmse)

1.0311080508924793


In [26]:
unique_movies = indexed.select('title_new').distinct()

In [27]:
unique_movies.count()

1664

In [28]:
a= unique_movies.alias('a')

In [29]:
user_id=85

In [30]:
#filtering movies watched bu userid=85
watched_movies = indexed.filter(indexed['userId']==user_id).select('title_new').distinct()
watched_movies.count()

287

In [31]:
b = watched_movies.alias('b')

In [32]:
total_movies = a.join(b, a.title_new==b.title_new, how='left')
total_movies.show(10, False)

+---------+---------+
|title_new|title_new|
+---------+---------+
|305.0    |305.0    |
|596.0    |null     |
|299.0    |null     |
|769.0    |null     |
|692.0    |null     |
|934.0    |null     |
|1051.0   |null     |
|496.0    |null     |
|558.0    |558.0    |
|170.0    |null     |
+---------+---------+
only showing top 10 rows



In [33]:
remaining_movies = total_movies.where(col("b.title_new").isNull()).select(a.title_new).distinct()
remaining_movies.count()

1377

In [34]:
remaining_movies = remaining_movies.withColumn("userId", lit(int(user_id)))
remaining_movies.show(10, False)

+---------+------+
|title_new|userId|
+---------+------+
|596.0    |85    |
|299.0    |85    |
|769.0    |85    |
|692.0    |85    |
|934.0    |85    |
|1051.0   |85    |
|496.0    |85    |
|170.0    |85    |
|184.0    |85    |
|576.0    |85    |
+---------+------+
only showing top 10 rows



In [35]:
remaining_movies.show(10, False)

+---------+------+
|title_new|userId|
+---------+------+
|596.0    |85    |
|299.0    |85    |
|769.0    |85    |
|692.0    |85    |
|934.0    |85    |
|1051.0   |85    |
|496.0    |85    |
|170.0    |85    |
|184.0    |85    |
|576.0    |85    |
+---------+------+
only showing top 10 rows



In [36]:
recommendations = rec_model.transform(remaining_movies).orderBy('prediction', ascending=False)
recommendations.show(5,False)

+---------+------+----------+
|title_new|userId|prediction|
+---------+------+----------+
|1347.0   |85    |5.4393845 |
|1114.0   |85    |5.302881  |
|1411.0   |85    |4.817681  |
|401.0    |85    |4.7020826 |
|658.0    |85    |4.6431327 |
+---------+------+----------+
only showing top 5 rows



In [37]:
movie_title = IndexToString(inputCol='title_new', outputCol='title', labels=model.labels)
final_recom = movie_title.transform(recommendations)
final_recom.show(10, False)

+---------+------+----------+-----------------------------------------------------------------+
|title_new|userId|prediction|title                                                            |
+---------+------+----------+-----------------------------------------------------------------+
|1347.0   |85    |5.4393845 |Angel Baby (1995)                                                |
|1114.0   |85    |5.302881  |Bitter Moon (1992)                                               |
|1411.0   |85    |4.817681  |Boys, Les (1997)                                                 |
|401.0    |85    |4.7020826 |Three Colors: Red (1994)                                         |
|658.0    |85    |4.6431327 |Return of Martin Guerre, The (Retour de Martin Guerre, Le) (1982)|
|1109.0   |85    |4.640209  |Unzipped (1995)                                                  |
|514.0    |85    |4.6125126 |Jean de Florette (1986)                                          |
|1342.0   |85    |4.606392  |8 Heads in 